In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models import Word2Vec

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define a function for text preprocessing (stemming and cleaning)
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text if not word in set(stopwords.words('english'))])
    return text

# Load dataset (adjust path as per your file location)
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/fakeNewsData.csv')
news_dataset = news_dataset.fillna('')

# Combine author and title into content
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']
news_dataset['content'] = news_dataset['content'].apply(preprocess_text)

# Tokenize text for Word2Vec training
tokenized_text = [text.split() for text in news_dataset['content']]

# Train Word2Vec model
wv_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Function to average Word2Vec vectors for a text
def word_averaging(wv_model, words):
    mean = np.zeros((wv_model.vector_size,))
    count = 0.
    for word in words:
        if word in wv_model.wv:
            mean += wv_model.wv[word]
            count += 1.
    if count != 0:
        mean /= count
    return mean

# Convert each document into average Word2Vec vectors
X = np.array([word_averaging(wv_model, words) for words in tokenized_text])
y = news_dataset['label']

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Predictions
y_pred = knn_model.predict(X_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from sklearn.metrics import roc_auc_score

# Predictions
y_pred = knn_model.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.4f}")

# Recall (Sensitivity)
recall = recall_score(y_test, y_pred)
print(f"Recall (Sensitivity): {recall:.4f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

# AUC Score
y_scores = knn_model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_scores)
print(f"AUC Score: {auc_score:.4f}")

# Calculate Specificity
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
specificity = TN / (TN + FP)
print(f"Specificity: {specificity:.4f}")


Confusion Matrix:
[[1920  212]
 [ 138 1890]]
Accuracy: 0.9159
Precision: 0.8991
Recall (Sensitivity): 0.9320
F1 Score: 0.9153
Matthews Correlation Coefficient (MCC): 0.8323
AUC Score: 0.9603
Specificity: 0.9006
